# Offline Engine API

SGLang provides a direct inference engine without the need for an HTTP server, especially for use cases where additional HTTP server adds unnecessary complexity or overhead. Here are two general use cases:

- Offline Batch Inference
- Custom Server on Top of the Engine

This document focuses on the offline batch inference, demonstrating four different inference modes:

- Non-streaming synchronous generation
- Streaming synchronous generation
- Non-streaming asynchronous generation
- Streaming asynchronous generation

Additionally, you can easily build a custom server on top of the SGLang offline engine. A detailed example working in a python script can be found in [custom_server](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/custom_server.py).

## Offline Batch Inference

SGLang offline engine supports batch inference with efficient scheduling.

In [1]:
# launch the offline engine

import sglang as sgl
import asyncio

llm = sgl.Engine(model_path="meta-llama/Meta-Llama-3.1-8B-Instruct")

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:00<00:02,  1.13it/s]


Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:01<00:01,  1.15it/s]


Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:02<00:00,  1.67it/s]


Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.38it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.36it/s]



...

 65%|██████▌   | 15/23 [00:03<00:01,  5.02it/s]

 74%|███████▍  | 17/23 [00:04<00:01,  5.07it/s]

 83%|████████▎ | 19/23 [00:04<00:00,  5.07it/s]

 91%|█████████▏| 21/23 [00:04<00:00,  5.12it/s]

100%|██████████| 23/23 [00:05<00:00,  4.33it/s]


### Non-streaming Synchronous Generation

In [2]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Hello, my name is
Generated text:  René. I am a Dutch man living in the Netherlands. I am very interested in mathematics, computers and technology in general. I am also a bit of a puzzle nut, so I love solving all sorts of puzzles and brain teasers. I am a member of the Dutch Sudoku Society and have solved many Sudoku puzzles. I have even been featured in the Dutch newspaper “Het Algemeen Dagblad” as the best Sudoku solver in the Netherlands.
In my free time I like to solve mathematical problems and riddles. I also like to tinker with my old computer, a 486DX-25 which I have upgraded to
Prompt: The president of the United States is
Generated text:  not, of course, the leader of the country in any formal sense; that is the function of the Vice President. The president is, however, the leader of the executive branch of the government, and thus has a great deal of power and influence. In addition, the president is often considered the leader of the country as a whole, particularly

### Streaming Synchronous Generation

In [3]:
prompts = [
    "Hello, my name is",
    "The capital of France is",
    "The future of AI is",
]
sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing synchronous streaming generation ===")

for prompt in prompts:
    print(f"\nPrompt: {prompt}")
    print("Generated text: ", end="", flush=True)

    for chunk in llm.generate(prompt, sampling_params, stream=True):
        print(chunk["text"], end="", flush=True)
    print()


=== Testing synchronous streaming generation ===

Prompt: Hello, my name is
Generated text: 

 Wesley

 and

 I

 am

...

uate

 and

 even

 amplify

 existing

### Non-streaming Asynchronous Generation

In [4]:
prompts = [
    "Hello, my name is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous batch generation ===")


async def main():
    outputs = await llm.async_generate(prompts, sampling_params)

    for prompt, output in zip(prompts, outputs):
        print(f"\nPrompt: {prompt}")
        print(f"Generated text: {output['text']}")


asyncio.run(main())


=== Testing asynchronous batch generation ===



Prompt: Hello, my name is
Generated text:  Nicole and I'm a Nutella fanatic. I'm a self-proclaimed Italian food expert, I've lived in Italy for years, and have become addicted to all things hazelnut and chocolate. I also love traveling, cooking, and sharing my passion for food with others. I'm excited to be here sharing my adventures and recipes with you!

Here are some popular posts to get you started:

*   [Nutella Stuffed French Toast](https://www.tasteofhome.com/recipes/nutella-stuffed-french-toast/): A decadent breakfast treat that combines the richness of Nutella with the fluffiness of

Prompt: The capital of France is
Generated text:  one of the most beautiful and romantic cities in the world. Paris is known for its stunning architecture, world-class museums, and charming neighborhoods. In this article, we will explore some of the best places to visit in Paris, including iconic landmarks, cultural attractions, and hidden gems.
The Eiffel Tower (La Tour Eiffel) is an iconic symb

### Streaming Asynchronous Generation

In [5]:
prompts = [
    "Hello, my name is",
    "The capital of France is",
    "The future of AI is",
]
sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous streaming generation ===")


async def main():
    for prompt in prompts:
        print(f"\nPrompt: {prompt}")
        print("Generated text: ", end="", flush=True)

        generator = await llm.async_generate(prompt, sampling_params, stream=True)
        async for chunk in generator:
            print(chunk["text"], end="", flush=True)
        print()


asyncio.run(main())


=== Testing asynchronous streaming generation ===

Prompt: Hello, my name is
Generated text: 

 Tim

 and

 I

 am

...

 could

 compromise

 sensitive

 information

 and

In [6]:
llm.shutdown()